<div dir="rtl">
       <h1>تبدیل فایل های IPYNB به HTML و استقرار خودکار در github-pages (بخش اول)</h1>
</div><br/>

<div dir="rtl">
       تو این پست قراره با نوشتن اسکریپت <strong>پایتون </strong>فایل های با فرمت <strong>ipynb </strong>رو به <strong>HTML </strong>تبدیل کنیم و به صورت خودکار با استفاده از <a href="https://github.com/features/actions" target="_blank" rel="noopener nofollow">github actions</a> تو <a href="https://pages.github.com/" target="_blank" rel="noopener nofollow">github pages</a> منتشرش کنیم.
</div><br/>

<div dir="rtl">
       <h2>فایل های با فرمت ipynb</h2>
</div><br/>

<div dir="rtl">
       فایل <strong>IPYNB </strong>یک سند نوت بوکه که توسط <a href="https://ipython.org/notebook.html" target="_blank" rel="noopener nofollow">jupyter notebook</a> ایجاد شده، یک محیط محاسباتی تعاملی که به ما تو دستکاری و تجزیه و تحلیل داده ها با استفاده از پایتون کمک می کنه.
</div><br/>

<div dir="rtl">
       <h2>اکشن های github</h2>
</div><br/>

<div dir="rtl">
       به کمک ویژگی <a href="https://github.com/features/actions" target="_blank" rel="noopener nofollow">github actions</a> خیلی از کارا رو مثل تست کد یا دیپلوی کردن کد جدید یا تو مورد ما، تبدیل فایل ها و انتقالشون به <a href="https://pages.github.com/" target="_blank" rel="noopener nofollow">github pages</a> رو به صورت خودکار میشه انجام داد.
</div><br/>

<hr style="background-color: floralwhite;" />

<br/><div dir="rtl">
    اسکریپتی که قراره بنویسیم رو میتونین از <a href="https://github.com/iranipy/iranipy" target="_blank" rel="noopener nofollow">مخزن اصلی گیت هاب ایرانی پای</a> بردارید.
    <ul><li><a href="https://github.com/iranipy/iranipy/blob/main/projects/ipynb_docs_renderer/script/ipynb_docs_renderer.py" target="_blank" rel="noopener nofollow">مشاهده اسکریپت</a></li><li><a href="https://github.com/iranipy/iranipy/tree/main/projects/ipynb_docs_renderer" target="_blank" rel="noopener nofollow">مشاهده پروژه</a> (شامل همه کدهای نوشته شده و فایلهای راهنما)</li></ul>
</div><br/>

<hr style="background-color: floralwhite;" />

<div dir="rtl">
       <h2>ایمپورت ماژول های استفاده شده</h2>
</div><br/>

In [1]:
import os
import pathlib
import shutil
import sys
from concurrent.futures import ThreadPoolExecutor

<br/><div dir="rtl">
    تو این قسمت ماژول های مورد نیاز رو ایمپورت میکنیم. همشون از لایبرری های استاندارد و built-in پایتون هستن، پس نیازی به نصب چیزی نیست.
</div><br/>

<div dir="rtl">
    <h2>تعریف متغیر های گلوبال</h2>
</div><br/>

In [2]:
_DOCS_PATH = 'docs'
_PROJECTS_PATH = 'projects'
_IPYNB_GLOB = '**/*.ipynb'
_GIT_MAIN_BRANCH_NAME = 'main'
_GIT_EMAIL = 'iranipy@users.noreply.github.com'
_GIT_USERNAME = 'iranipy'
_GIT_COMMIT_MESSAGE = 'new rendered docs added by ipynb_docs_renderer action'
_CONVERTED_FILE_FORMAT = 'html'
_CONVERT_COMMAND = f'jupyter nbconvert --to {_CONVERTED_FILE_FORMAT}'
_DEPENDENCIES = ['jupyterlab', 'nbconvert']

<br/><div dir="rtl">
    اینا متغیر هایی هستن که ادامه کار بهشون نیاز داریم و مقدارشون تغییری نمیکنه. مقادیر رو به خواست خودتون تغییر بدین، مثلا مقدار <strong>GIT_EMAIL_ </strong>رو به ایمیل حساب خودتون تغییر بدین. دلیل استفاده از حروف بزرگ و آندرلاین قبل اسمشون اینه که میخوایم نشون بدیم <strong>ثابت (constant)</strong> هستن و <strong>خصوصی (private)</strong> اما از اونجایی که تو پایتون این ویژگی ها به صورت داخلی و تو سطح <strong>مفسر (interpreter)</strong> وجود نداره، این موضوع فقط به صورت قرار دادیه؛ یعنی شما هرجای کد که بخواید میتونید مقدارشون رو عوض یا تو هر ماژولی ایمپورتشون کنین. ولی خب نکنین :)
</div><br/>

<div dir="rtl">
    ماژول های جدیدی که تو این بخش به عنوان <strong>وابستگی (dependency)</strong> دیده میشه <a href="https://nbconvert.readthedocs.io/en/latest/index.html" target="_blank" rel="noopener nofollow">nbconvert</a>و <a href="https://jupyterlab.readthedocs.io/en/stable/" target="_blank" rel="noopener nofollow">jupyterlab</a>هستن که نیازی به نصبشون نیست و در ادامه بهشون میرسیم.
</div><br/>

<div dir="rtl">
    <h2>نصب پیش نیاز ها روی ماشین</h2>
</div><br/>

In [3]:
def _install_dependencies():
    with ThreadPoolExecutor() as executer:
        executer.map(os.system, [f'pip install {d}' for d in _DEPENDENCIES])

<br/><div dir="rtl">
    وظیفه این تابع نصب پیش نیاز های برنامه (اعضای لیست <strong>DEPENDENCIES_</strong>) تو محیط اجرای برنامه ست.
</div><br/>

<div dir="rtl">
    از کلاس <a href="https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ThreadPoolExecutor" target="_blank" rel="noopener nofollow">ThreadPoolExecutor</a> استفاده میکنیم. متغیر <strong>executer </strong>یک <strong>instance </strong>از <a href="https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ThreadPoolExecutor" target="_blank" rel="noopener nofollow">ThreadPoolExecutor</a> حساب میشه. (آشنایی بیشتر با <a href="https://realpython.com/python-with-statement/" target="_blank" rel="noopener nofollow">context manager</a>)
</div><br/>

<div dir="rtl">
    تو خط سوم از یکی از متد های <a href="https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ThreadPoolExecutor" target="_blank" rel="noopener nofollow">ThreadPoolExecutor</a> به اسم <strong>map </strong>استفاده کردیم. عملکرد این متد دقیقا مثل تابع <a href="https://docs.python.org/3/library/functions.html#map" target="_blank" rel="noopener nofollow">map</a> خود پایتونه. یعنی یک تابع و یک <strong>iterable </strong>به عنوان پارامتر میگیره و اون تابع رو روی تک تک اعضای اون <strong>iterable </strong>اجرا میکنه ولی فرقش اینه که هر عضو اون <strong>iterable </strong>رو تو یک <strong>رشته (Thread)</strong> جداگونه اجرا می کنه. تابعی که اینجا استفاده شده <strong>system </strong>از کتابخونه <strong>os </strong>هستش. این تابع دستوری که به شکل <strong>رشته (string)</strong> به عنوان یک پارامتر بهش میدید رو تو یک <strong>subshell </strong>اجرا میکنه.
</div><br/>

<div dir="rtl">
    <h2>تبدیل ipynb به HTML و انتقال به GitHub Pages</h2>
</div><br/>

In [4]:
def _render_ipynb(ipynb: list):
    directory, file = os.path.split(ipynb)
    rendered_file = f'{os.path.splitext(file)[0]}.{_CONVERTED_FILE_FORMAT}'
    rendered_file_path = os.path.join(directory, rendered_file)
    os.system(f'{_CONVERT_COMMAND} {ipynb.absolute()}')
    shutil.move(rendered_file_path, os.path.join(_DOCS_PATH, rendered_file))

def _run_rendering():
    IPYNB_DOCS = list(pathlib.Path(_PROJECTS_PATH).glob(_IPYNB_GLOB))
    with ThreadPoolExecutor() as executer:
        executer.map(_render_ipynb, IPYNB_DOCS)

<br/><div dir="rtl">
    تو تابع <strong>render_ipynb_ </strong>یک ابجکت <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path" target="_blank" rel="noopener nofollow">pathlib.Path</a> به عنوان پارامتر دریافت میشه و عملیات تبدیل فرمت و انتقالش به دایرکتوری مورد نظر (معادل با مقدار <strong>DOCS_PATH_</strong>) انجام میشه.
</div><br/>

<div dir="rtl">
    <ol><li>تو خط اول با استفاده از <a href="https://docs.python.org/3/library/os.path.html#os.path.split" target="_blank" rel="noopener nofollow">os.path.split</a> مسیر فایل و اسم فایل رو به ترتیب به متغیرهای <strong>directory </strong>و <strong>file </strong>نسبت دادیم.</li><li>تو خط دوم اسم فایل نهایی رو مشخص کردیم. این اسم متشکل از اسم فایلی که در حال حاضر روش کار میکنیم و پسوند <strong>html </strong>(معادل با مقدار <strong>CONVERTED_FILE_FORMAT_ </strong>) هستش.</li><li>تو خط سوم مسیر نهایی فایل <strong>HTML </strong>مورد نظر رو با <a href="https://docs.python.org/3/library/os.path.html#os.path.join" target="_blank" rel="noopener nofollow">os.path.join</a> میسازیم. دلیل استفاده از این متد به جای هارد کد کردن اینه که این متد با توجه به سیستم عامل از <strong>separator </strong>مناسب استفاده میکنه و از ارور جلوگیری میکنه.</li><li>تو خط چهارم از تابع <strong>system </strong>استفاده شده که تو بخش های قبلی عملکردش توضیح داده شد، که اینجا داره دستور معادل با مقدار <strong>CONVERT_COMMAND_ </strong>رو با یک پارامتر با مقدار حاصل از فراخوانی <strong>ipynb.absolute</strong> اجرا میکنه. این متد <a href="https://en.wikipedia.org/wiki/Path_(computing)" target="_blank" rel="noopener nofollow">مسیر مطلق</a> فایل رو بر میگردونه.</li><li>خط آخر فایل تولید شده خط بالایی رو از مسیر فعلیش برمیداره و به دایرکتوری مقصد (که اینجا <strong>docs </strong>در نظر گرفته شده) منتقل میکنه. این متد بدون نوشتن اسم فایل هم کار میکنه، ینی اگه فقط اسم دایرکتوری رو مینوشیتم هم درست بود ولی نکته منفی ای که ننوشتن اسم فایل مقصد داره اینه که اگه فایلی با اسم مشابه فایل مبدا وجود داشته باشه به جای بازنویسی کردنش ارور میده.</li><li>تابع <strong>run_rendering_ </strong>دوتا وظیفه داره. اول درست کردن لیستی از تمام فایل های با فرمت <strong>ipynb </strong>توی دایرکتوری مورد نظر (که اینجا <strong>projects </strong>در نظر گرفته شده) و تبدیل اونا به <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path" target="_blank" rel="noopener nofollow">pathlib.Path</a> و بعد اجرای تابع <strong>render_ipynb_ </strong>روی اعضای لیست به صورت همزمان.</li></ol>
</div><br/>

<div dir="rtl">
    <h2>پوش کردن تغییرات صورت گرفته روی مخزن (repository) مورد نظر</h2>
</div><br/>

In [5]:
def _commit_push():
    commands = [
        f'git config --global user.email "{_GIT_EMAIL}" && git config --global user.name "{_GIT_USERNAME}"',
        'git add .',
        f'git commit -m "{_GIT_COMMIT_MESSAGE}"',
        f'git fetch origin {_GIT_MAIN_BRANCH_NAME}',
        f'git push origin {_GIT_MAIN_BRANCH_NAME}',
    ]
    list(map(os.system, commands))

<br/><div dir="rtl">
    تو این تابع تمام دستوراتی که برای کامیت و پوش کردن لازم هست رو یک لیست قرار دادیم و با استفاده از <a href="https://docs.python.org/3/library/functions.html#map" target="_blank" rel="noopener nofollow">map</a> تابع <strong>os.system</strong> روی اعضای لیست اجرا کردیم. دلیل استفاده از تابع <a href="https://docs.python.org/3/library/functions.html#func-list" target="_blank" rel="noopener nofollow">list</a> این هستش که اگه این کارو انجام ندیم تنها اتفاقی که میوفته اینه که تابع <a href="https://docs.python.org/3/library/functions.html#map" target="_blank" rel="noopener nofollow">map</a> یک <strong>map object</strong> بهمون میده و هیچکدوم از دستورات رو اجرا نمیکنه. به عبارت دیگه برای اجرا این دستورات باید روی <strong>map object، </strong>یک <strong>iterate </strong>بزنیم (به هرشکلی، مهم نیست).
</div><br/>

<div dir="rtl">
    <h2>رابط خط فرمان (CLI)</h2>
</div><br/>

In [6]:
def main():
    task = {
        'install-dependencies': _install_dependencies,
        'render': _run_rendering,
        'commit-push': _commit_push,
    }[sys.argv[1]]
    
    task()

<br/><div dir="rtl">
    تو این تابع یک دیکشنری داریم که تو این دیکشنری به هر کدوم از تابع هایی که بالاتر تعریف کردیم یک کلیدواژه اختصاص داده شده. هر کدوم از این کلیدواژه ها که موقع اجرای ماژول به عنوان آرگومان خط فرمان وارد بشه، برنامه همون تابع رو اجرا میکنه؛ در واقع روشیه برای شبیه سازی عبارت <strong>swicth-case</strong> موجود تو سایر زبان های برنامه نویسی. (<a href="https://docs.python.org/3/library/sys.html#sys.argv" target="_blank" rel="noopener nofollow">sys.argv</a>)
</div><br/>

<div dir="rtl">
    به عنوان مثال، اینجوری میتونیم تابع <strong>run_rendering_ </strong>رو اجرا کنیم:
</div><br/>

``python ipynb_docs_renderer.py render``

<br/><div dir="rtl">
    <h2>اجرای تابع اصلی</h2>
</div><br/>

In [8]:
if __name__ == '__main__':
    main()

KeyError: '--ip=127.0.0.1'

<br/><div dir="rtl">
    <a href="https://docs.python.org/3/library/__main__.html" target="_blank" rel="noopener nofollow">توضیحات بیشتر در مورد __main__</a>
</div><br/>

<hr style="background-color: floralwhite;" />

<br/><div dir="rtl">
    پایان بخش اول ....
</div><br/>

<div dir="rtl">
    امیدوارم از اولین بخش این پست استفاده کافی رو برده باشید. اگه شما هم مثل ما از خوندن مطالب پایتونی لذت میبرید، خوشحال میشیم ما رو تو شبکه های اجتماعی دنبال و در صورت داشتن انتقاد و پیشنهاد ما رو ازش مطلع کنید. لایک هم یادتون نره :)
</div><br/>

<div dir="rtl">
    <a href="https://vrgl.ir/tUpiw" target="_blank" rel="noopener nofollow">برای خوندن بخش دوم کلیک کنید.</a>
</div><br/>

<hr style="background-color: floralwhite;" />

<br/><div dir="rtl">
    <h4><strong>حمایت از ما</strong></h4>
</div><br/>

<div dir="rtl">
    با دنبال کردن ما در
</div>

<div dir="rtl">
    <ul><li><a href="https://github.com/iranipy" target="_blank" rel="noopener nofollow">github</a></li><li><a href="https://t.me/iranipy" target="_blank" rel="noopener nofollow">telegram</a></li><li><a href="https://instagram.com/iranipy" target="_blank" rel="noopener nofollow">instagram</a> </li><li><a href="https://twitter.com/_iranipy" target="_blank" rel="noopener nofollow">twitter</a> </li></ul>
</div>

<div dir="rtl">
    یا کمک مالی به مبلغ دلخواه از طریق <a href="https://idpay.ir/iranipy" target="_blank" rel="noopener nofollow">درگاه آیدی پی</a> میتونید از ما حمایت کنید.
</div>

<div dir="rtl">
    <strong>وب سایت (به زودی):</strong> <a href="https://iranipy.ir/" target="_blank" rel="noopener nofollow">iranipy.ir</a>
</div>